In [22]:
import os
from cnnClassifier.constants import *
from cnnClassifier.utils.comon import read_yaml, create_directories
import tensorflow as tf


%pwd

os.chdir("../")


%pwd



# Change to your project directory where config.yaml exists
os.chdir('C:/Users/shuda/End-to-end-with-MLFLOW')
print(f"New working directory: {os.getcwd()}")

import os
from cnnClassifier.constants import *
from cnnClassifier.utils.comon import read_yaml, create_directories  
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_dir: Path  # Path to training data
    validation_data_dir: Path  # Path to validation data
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


class ConfigurationManager:
    def __init__(self,
                  config_filepath=CONFIG_FILE_PATH, 
                  params_filepath=PARAMS_FILE_PATH):

       


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data_dir=Path(training.training_data_dir),  # Direct path to training data
            validation_data_dir=Path(training.validation_data_dir),  # Direct path to validation data
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale=1./255)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data_dir,  # Use validation data directory
            shuffle=False,
            **dataflow_kwargs
        )

        # Training data generator
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data_dir,  # Use training data directory
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()

    training.train()

except FileNotFoundError as fnf_error:
    print(f"FileNotFoundError: {fnf_error}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise

New working directory: C:\Users\shuda\End-to-end-with-MLFLOW
[2024-09-16 11:59:21,680: INFO: comon: yaml file: config\config.yaml loaded successfully]
[2024-09-16 11:59:21,687: INFO: comon: yaml file: params.yaml loaded successfully]
[2024-09-16 11:59:21,690: INFO: comon: created directory at: artifacts]
[2024-09-16 11:59:21,695: INFO: comon: created directory at: artifacts\training]
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
Epoch 1/3
38/38 [==============================] - 135s 4s/step - loss: 11.1871 - accuracy: 0.3886 - val_loss: 7.3463 - val_accuracy: 0.4531
Epoch 2/3
38/38 [==============================] - 131s 3s/step - loss: 5.0819 - accuracy: 0.5327 - val_loss: 3.7757 - val_accuracy: 0.5000
Epoch 3/3
38/38 [==============================] - 110s 3s/step - loss: 4.5478 - accuracy: 0.5662 - val_loss: 3.6819 - val_accuracy: 0.5781
